In [ ]:
from selenium import webdriver
import time
import pandas as pd
import requests
import sys
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime,timedelta
import time
import random
import re
import operator

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/raul.castillo/.wdm/drivers/chromedriver/mac64/101.0.4951.41]


In [ ]:
browser.get('https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Reviews-Playa_San_Pancho-San_Francisco_Pacific_Coast.html')

In [ ]:
data = pd.DataFrame(columns=('id','fecha_estancia','usuario','titulo','opinion','calificacion','genero','estado','lugar','x','y'))

In [ ]:
elem = browser.find_element_by_xpath("//*")
soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')

In [ ]:
rev = soup.find_all('div',"ffbzW _c")[0].get_text()

In [ ]:
len(rev)

295

In [ ]:
total = range(0,70,8) # En un rango de 0 a 10 reseñas, va a saltar de 5 en 5

In [ ]:
def clean_text(txt):
    txt = re.sub("[^a-záéíóúñüäë]", " ", txt.lower())
    # Nos pasa el textoo a minúsculas.
    txt = re.sub(' +',' ', txt)
    # Limpia nuestro texto.
    return txt.strip().split()
    # Nos regresa el texto con las delimitaciones especificadas.

def df_to_dict(df, key_column, val_column):
    """convierte dos pandas series en un diccionario"""
    xkey = df[key_column].tolist()
    xval = df[val_column].tolist()
    return dict(zip(xkey,xval))

def get_gender2(names):
    names = clean_text(names)
    names = [x for x in names if gender_list.get(x,'a') != 'a']
    gender ={'m':0, 'f':0, 'a':0}
    for i, name in enumerate(names):
        g = gender_list.get(name,'a')
        gender[g] += 1
        gender[g] += 2 if len(names) > 1 and i == 0 and g != 'a' else 0 
    gender['a'] = 0 if (gender['f']+gender['m']) > 0 else 1
    return max(gender.items(), key=operator.itemgetter(1))[0]

gender_list = pd.read_csv('nombres.csv')
gender_list = df_to_dict(gender_list, key_column='nombre', val_column='genero')

In [ ]:
url1 = "https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Reviews-"
url2 = "Playa_San_Pancho-San_Francisco_Pacific_Coast.html"

In [ ]:
browser.maximize_window()
for p in total:
    if p == 0:
        centro = '-'
    else:
        centro = '-or'+str(p)+'-'
    url = url1+centro+url2
    print(url1+centro+url2)
    #print(url)
    browser.get(url)
    
    desplazar = random.randint(400, 1200)
    browser.execute_script('window.scrollTo(1, '+str(desplazar)+');')
    time.sleep(2)
    
    elem = browser.find_element_by_xpath("//*")
    soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')
    samples = soup.find_all('div',"ffbzW _c")
    c=0
    for s in samples:
        ids = p+c+1
        sample = samples[c]
        autor = sample.find_all("span",'WlYyy cPsXC dTqpp')[0].get_text()
        gen = get_gender2(autor)
        cal = str(sample.find_all('svg',"RWYkj d H0")[0])
        cal = str(cal[17:18])
        titulo = sample.find_all('span',"NejBf")[0].get_text()
        opinion = sample.find_all('div',"WlYyy diXIH dDKKM")[0].get_text()
        fecha_est = sample.find_all('div',"WlYyy diXIH cspKb bQCoY")[0].get_text()
        fecha_est = fecha_est[11:]
        #data = data.append({'fecha_estancia': fecha_est,'usuario':nombre,'calificacion':cal,'titulo':titulo,'opinion':opinion},ignore_index=True)
        data = data.append({'id':ids, 'fecha_estancia': fecha_est,'usuario':autor,'calificacion':cal,'titulo':titulo,'opinion':opinion,'genero':gen,'estado':'Nayarit','lugar':'Playa San Pancho','x':'20.9022451','y':'-105.4159466'},ignore_index=True)
        c = c+1
    
    p+5
    sleep = random.randint(5, 15)
    print(sleep)
    time.sleep(sleep)
          
print('Ya puedes exportar')

https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Reviews--Playa_San_Pancho-San_Francisco_Pacific_Coast.html
6
https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Reviews--or8-Playa_San_Pancho-San_Francisco_Pacific_Coast.html
12
https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Reviews--or16-Playa_San_Pancho-San_Francisco_Pacific_Coast.html
9
https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Reviews--or24-Playa_San_Pancho-San_Francisco_Pacific_Coast.html
12
https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Reviews--or32-Playa_San_Pancho-San_Francisco_Pacific_Coast.html
15
https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Reviews--or40-Playa_San_Pancho-San_Francisco_Pacific_Coast.html
6
https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Reviews--or48-Playa_San_Pancho-San_Francisco_Pacific_Coast.html
10
https://www.tripadvisor.com.mx/Attraction_Review-g552077-d12653697-Revi

In [ ]:
data

,id,fecha_estancia,usuario,titulo,opinion,calificacion,genero,estado,lugar,x,y
0,1,24 de marzo de 2022,María Julieta E,Playa amplia,La playa es muy amplia y se puede caminar muy ...,5,f,Nayarit,Playa San Pancho,20.9022451,-105.4159466
1,2,17 de abril de 2021,Gerardo R,San Pancho,"De verdad que no le pide nada a otras playas, ...",5,m,Nayarit,Playa San Pancho,20.9022451,-105.4159466
2,3,15 de marzo de 2021,Diego G,Excelente lugar!!,Guarda la magia de un pequeño pueblo con una p...,5,m,Nayarit,Playa San Pancho,20.9022451,-105.4159466
3,4,9 de febrero de 2021,Maquiro,Muy buena playa,"Excelente playa, tranquila para meterse a nada...",4,a,Nayarit,Playa San Pancho,20.9022451,-105.4159466
4,5,4 de enero de 2021,Odyssey585290,preciosa playa,lastima que la gente deje la basura ahi botada...,4,a,Nayarit,Playa San Pancho,20.9022451,-105.4159466
...,...,...,...,...,...,...,...,...,...,...,...
80,65,,David,"Pequeña, pintoresca y hermosa puesta de sol.","Fui aquí en noviembre de 2020. Linda playa, no...",4,m,Nayarit,Playa San Pancho,20.9022451,-105.4159466
81,66,,Kate H,"Esta es una playa hermosa y tranquila, pero no...",Si quieres tu mejor oportunidad de nadar de fo...,5,f,Nayarit,Playa San Pancho,20.9022451,-105.4159466
82,67,,Quirr0,Impresionante playa en la Riviera Nayarita,Uno de mis lugares favoritos y playas en la Ri...,5,a,Nayarit,Playa San Pancho,20.9022451,-105.4159466
83,68,,Mike T,Limpio y no estaba lleno!,Nos alojábamos en Sayulita y buscando una play...,5,m,Nayarit,Playa San Pancho,20.9022451,-105.4159466


In [ ]:
data.groupby(['calificacion']).count()

,id,fecha_estancia,usuario,titulo,opinion,genero,estado,lugar,x,y
calificacion,,,,,,,,,,
2,2,2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3,3,3
4,23,23,23,23,23,23,23,23,23,23
5,57,57,57,57,57,57,57,57,57,57


In [ ]:
export_csv = data.to_csv ("MDC_17.csv", index = None, header=True , encoding='UTF-8')

In [ ]:
c=0
samples = soup.find_all("div", "ffbzW _c")
sample = samples[c]

In [ ]:
samples

[<div class="ffbzW _c" data-automation="reviewCard"><div class="ezLVz f M k"><div class="bJyQA f u o"><div class="fksET"><div class="baYaB f u"><div class="bQEAh" style="z-index:0"><div class=""><div class="fknJY svvjk Rb I o"><div class=""><a aria-hidden="true" class="iPqaD _F G- ddFHE eKwUx" href="/Profile/iqueabel" tabindex="-1" target="_self"><div class="btBEK eglJy"><picture class="dugSS _R" style="width:32px;height:32px"><img alt="Abel Gómez" height="100" loading="lazy" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/22/65/93/59/abel-m.jpg?w=100&amp;h=-1&amp;s=1" width="100"/></picture></div></a></div></div></div></div></div></div><div class="cjhIj"><span class="WlYyy cPsXC dTqpp"><a class="iPqaD _F G- ddFHE eKwUx btBEK fUpii" href="/Profile/iqueabel" target="_self">Abel Gómez</a></span><div class="ddOtn"><div class="WlYyy diXIH bQCoY"><span>Reynosa, México</span><span class="fhriQ">13 aportes</span></div></div></div></div><div class="Ixpqj f"><button aria-label="Haz

In [ ]:
len(sample)

8

In [ ]:
autor = sample.find_all("span",'WlYyy cPsXC dTqpp')[0].get_text()

In [ ]:
nombre

'Abel Gómez'

In [ ]:
cal = str(sample.find_all('svg',"RWYkj d H0")[0])
cal = str(cal[17:18])

In [ ]:
cal

'5'

In [ ]:
titulo = sample.find_all('span',"NejBf")[0].get_text()

In [ ]:
titulo

'Playa Hermosa'

In [ ]:
opinion = sample.find_all('div',"WlYyy diXIH dDKKM")[0].get_text()

In [ ]:
opinion

'La playa principal de Cabo San Lucas, es su carta de presentación, aquí encuentras fiesta, tragos, beach club’s, restaurantes excelentes con distintas opciones culinarias para degustar, varios hoteles de renombre a pie de playa y Medano es de las pocas playas nadables en Cabo.'

In [ ]:
fecha_est = sample.find_all('div',"WlYyy diXIH cspKb bQCoY")[0].get_text()
fecha_est = fecha_est[11:]

In [ ]:
fecha_est

'9 de marzo de 2022'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a4b48cf0-c50a-44f8-990b-e0da28b0964f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>